In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [2]:
counter = 0

In [19]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,30):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2009","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
data
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


['{"get":"players","parameters":{"league":"39","page":"1","season":"2010"},"errors":[],"results":20,"paging":{"current":1,"total":34},"response":[{"player":{"id":468,"name":"V. \\u0106orluka","firstname":"Vedran","lastname":"\\u0106orluka","age":35,"birth":{"date":"1986-02-05","place":"Derventa","country":"Bosnia and Herzegovina"},"nationality":"Croatia","height":"192 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/468.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2010},"games":{"appearences":15,"lineups":13,"minutes":1166,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":2,"out":3,"bench":16},"shots":{"total":null,"on":nu

In [20]:
counter

65

In [21]:
#complete year array
for i in range(30,33):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2009","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)
    
data

['{"get":"players","parameters":{"league":"39","page":"1","season":"2010"},"errors":[],"results":20,"paging":{"current":1,"total":34},"response":[{"player":{"id":468,"name":"V. \\u0106orluka","firstname":"Vedran","lastname":"\\u0106orluka","age":35,"birth":{"date":"1986-02-05","place":"Derventa","country":"Bosnia and Herzegovina"},"nationality":"Croatia","height":"192 cm","weight":"84 kg","injured":false,"photo":"https:\\/\\/media.api-sports.io\\/football\\/players\\/468.png"},"statistics":[{"team":{"id":47,"name":"Tottenham","logo":"https:\\/\\/media.api-sports.io\\/football\\/teams\\/47.png"},"league":{"id":39,"name":"Premier League","country":"England","logo":"https:\\/\\/media.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media.api-sports.io\\/flags\\/gb.svg","season":2010},"games":{"appearences":15,"lineups":13,"minutes":1166,"number":null,"position":"Defender","rating":null,"captain":false},"substitutes":{"in":2,"out":3,"bench":16},"shots":{"total":null,"on":nu

In [22]:
from json import loads
#Data2020 is complete, next step is to decide how to model and keep which part of the data
#data2020Json = list(map(lambda i: data2020[i].json(),data2020))
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

34

In [24]:
len(jsonData[33]['response'])

3

In [25]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [26]:
#finalPageItems
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
finalPageItems

3

In [27]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

#To change before every extraction
iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            print(i,j,pkey)
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

0 0 id
0 0 firstname
0 0 lastname
0 0 age
0 0 nationality
0 0 height
0 0 weight
0 0 injured
0 1 id
0 1 firstname
0 1 lastname
0 1 age
0 1 nationality
0 1 height
0 1 weight
0 1 injured
0 2 id
0 2 firstname
0 2 lastname
0 2 age
0 2 nationality
0 2 height
0 2 weight
0 2 injured
0 3 id
0 3 firstname
0 3 lastname
0 3 age
0 3 nationality
0 3 height
0 3 weight
0 3 injured
0 4 id
0 4 firstname
0 4 lastname
0 4 age
0 4 nationality
0 4 height
0 4 weight
0 4 injured
0 5 id
0 5 firstname
0 5 lastname
0 5 age
0 5 nationality
0 5 height
0 5 weight
0 5 injured
0 6 id
0 6 firstname
0 6 lastname
0 6 age
0 6 nationality
0 6 height
0 6 weight
0 6 injured
0 7 id
0 7 firstname
0 7 lastname
0 7 age
0 7 nationality
0 7 height
0 7 weight
0 7 injured
0 8 id
0 8 firstname
0 8 lastname
0 8 age
0 8 nationality
0 8 height
0 8 weight
0 8 injured
0 9 id
0 9 firstname
0 9 lastname
0 9 age
0 9 nationality
0 9 height
0 9 weight
0 9 injured
0 10 id
0 10 firstname
0 10 lastname
0 10 age
0 10 nationality
0 10 height
0 10 

7 19 height
7 19 weight
7 19 injured
8 0 id
8 0 firstname
8 0 lastname
8 0 age
8 0 nationality
8 0 height
8 0 weight
8 0 injured
8 1 id
8 1 firstname
8 1 lastname
8 1 age
8 1 nationality
8 1 height
8 1 weight
8 1 injured
8 2 id
8 2 firstname
8 2 lastname
8 2 age
8 2 nationality
8 2 height
8 2 weight
8 2 injured
8 3 id
8 3 firstname
8 3 lastname
8 3 age
8 3 nationality
8 3 height
8 3 weight
8 3 injured
8 4 id
8 4 firstname
8 4 lastname
8 4 age
8 4 nationality
8 4 height
8 4 weight
8 4 injured
8 5 id
8 5 firstname
8 5 lastname
8 5 age
8 5 nationality
8 5 height
8 5 weight
8 5 injured
8 6 id
8 6 firstname
8 6 lastname
8 6 age
8 6 nationality
8 6 height
8 6 weight
8 6 injured
8 7 id
8 7 firstname
8 7 lastname
8 7 age
8 7 nationality
8 7 height
8 7 weight
8 7 injured
8 8 id
8 8 firstname
8 8 lastname
8 8 age
8 8 nationality
8 8 height
8 8 weight
8 8 injured
8 9 id
8 9 firstname
8 9 lastname
8 9 age
8 9 nationality
8 9 height
8 9 weight
8 9 injured
8 10 id
8 10 firstname
8 10 lastname
8 10 a

16 4 age
16 4 nationality
16 4 height
16 4 weight
16 4 injured
16 5 id
16 5 firstname
16 5 lastname
16 5 age
16 5 nationality
16 5 height
16 5 weight
16 5 injured
16 6 id
16 6 firstname
16 6 lastname
16 6 age
16 6 nationality
16 6 height
16 6 weight
16 6 injured
16 7 id
16 7 firstname
16 7 lastname
16 7 age
16 7 nationality
16 7 height
16 7 weight
16 7 injured
16 8 id
16 8 firstname
16 8 lastname
16 8 age
16 8 nationality
16 8 height
16 8 weight
16 8 injured
16 9 id
16 9 firstname
16 9 lastname
16 9 age
16 9 nationality
16 9 height
16 9 weight
16 9 injured
16 10 id
16 10 firstname
16 10 lastname
16 10 age
16 10 nationality
16 10 height
16 10 weight
16 10 injured
16 11 id
16 11 firstname
16 11 lastname
16 11 age
16 11 nationality
16 11 height
16 11 weight
16 11 injured
16 12 id
16 12 firstname
16 12 lastname
16 12 age
16 12 nationality
16 12 height
16 12 weight
16 12 injured
16 13 id
16 13 firstname
16 13 lastname
16 13 age
16 13 nationality
16 13 height
16 13 weight
16 13 injured
16 14

20 8 weight
20 8 injured
20 9 id
20 9 firstname
20 9 lastname
20 9 age
20 9 nationality
20 9 height
20 9 weight
20 9 injured
20 10 id
20 10 firstname
20 10 lastname
20 10 age
20 10 nationality
20 10 height
20 10 weight
20 10 injured
20 11 id
20 11 firstname
20 11 lastname
20 11 age
20 11 nationality
20 11 height
20 11 weight
20 11 injured
20 12 id
20 12 firstname
20 12 lastname
20 12 age
20 12 nationality
20 12 height
20 12 weight
20 12 injured
20 13 id
20 13 firstname
20 13 lastname
20 13 age
20 13 nationality
20 13 height
20 13 weight
20 13 injured
20 14 id
20 14 firstname
20 14 lastname
20 14 age
20 14 nationality
20 14 height
20 14 weight
20 14 injured
20 15 id
20 15 firstname
20 15 lastname
20 15 age
20 15 nationality
20 15 height
20 15 weight
20 15 injured
20 16 id
20 16 firstname
20 16 lastname
20 16 age
20 16 nationality
20 16 height
20 16 weight
20 16 injured
20 17 id
20 17 firstname
20 17 lastname
20 17 age
20 17 nationality
20 17 height
20 17 weight
20 17 injured
20 18 id
20

24 11 height
24 11 weight
24 11 injured
24 12 id
24 12 firstname
24 12 lastname
24 12 age
24 12 nationality
24 12 height
24 12 weight
24 12 injured
24 13 id
24 13 firstname
24 13 lastname
24 13 age
24 13 nationality
24 13 height
24 13 weight
24 13 injured
24 14 id
24 14 firstname
24 14 lastname
24 14 age
24 14 nationality
24 14 height
24 14 weight
24 14 injured
24 15 id
24 15 firstname
24 15 lastname
24 15 age
24 15 nationality
24 15 height
24 15 weight
24 15 injured
24 16 id
24 16 firstname
24 16 lastname
24 16 age
24 16 nationality
24 16 height
24 16 weight
24 16 injured
24 17 id
24 17 firstname
24 17 lastname
24 17 age
24 17 nationality
24 17 height
24 17 weight
24 17 injured
24 18 id
24 18 firstname
24 18 lastname
24 18 age
24 18 nationality
24 18 height
24 18 weight
24 18 injured
24 19 id
24 19 firstname
24 19 lastname
24 19 age
24 19 nationality
24 19 height
24 19 weight
24 19 injured
25 0 id
25 0 firstname
25 0 lastname
25 0 age
25 0 nationality
25 0 height
25 0 weight
25 0 inju

28 15 firstname
28 15 lastname
28 15 age
28 15 nationality
28 15 height
28 15 weight
28 15 injured
28 16 id
28 16 firstname
28 16 lastname
28 16 age
28 16 nationality
28 16 height
28 16 weight
28 16 injured
28 17 id
28 17 firstname
28 17 lastname
28 17 age
28 17 nationality
28 17 height
28 17 weight
28 17 injured
28 18 id
28 18 firstname
28 18 lastname
28 18 age
28 18 nationality
28 18 height
28 18 weight
28 18 injured
28 19 id
28 19 firstname
28 19 lastname
28 19 age
28 19 nationality
28 19 height
28 19 weight
28 19 injured
29 0 id
29 0 firstname
29 0 lastname
29 0 age
29 0 nationality
29 0 height
29 0 weight
29 0 injured
29 1 id
29 1 firstname
29 1 lastname
29 1 age
29 1 nationality
29 1 height
29 1 weight
29 1 injured
29 2 id
29 2 firstname
29 2 lastname
29 2 age
29 2 nationality
29 2 height
29 2 weight
29 2 injured
29 3 id
29 3 firstname
29 3 lastname
29 3 age
29 3 nationality
29 3 height
29 3 weight
29 3 injured
29 4 id
29 4 firstname
29 4 lastname
29 4 age
29 4 nationality
29 4 h

32 18 firstname
32 18 lastname
32 18 age
32 18 nationality
32 18 height
32 18 weight
32 18 injured
32 19 id
32 19 firstname
32 19 lastname
32 19 age
32 19 nationality
32 19 height
32 19 weight
32 19 injured
33 0 id
33 0 firstname
33 0 lastname
33 0 age
33 0 nationality
33 0 height
33 0 weight
33 0 injured
33 1 id
33 1 firstname
33 1 lastname
33 1 age
33 1 nationality
33 1 height
33 1 weight
33 1 injured
33 2 id
33 2 firstname
33 2 lastname
33 2 age
33 2 nationality
33 2 height
33 2 weight
33 2 injured


In [28]:
len(finalArr)

663

In [29]:
#TO NOTE: Year in query is season XXXX/XXXX+1
#Example if 2020 is past as param, the data returned is from the season 2020/2021
#Create a 2d array
arr = np.array(finalArr)
arr

array([[468, 'Vedran', 'Ćorluka', ..., None, None, None],
       [5982, 'Carlos Alberto', 'Tevez', ..., None, None, None],
       [15441, 'Abdisalam', 'Ibrahim', ..., None, None, None],
       ...,
       [2793, 'Guðlaugur Victor', 'Pálsson', ..., None, None, None],
       [18829, 'Jack', 'Wilshere', ..., None, None, None],
       [48368, 'Martin', 'Hansen', ..., None, None, None]], dtype=object)

In [30]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

id          firstname     lastname age nationality  height weight  \
0       468             Vedran      Ćorluka  35     Croatia  192 cm  84 kg   
1      5982     Carlos Alberto        Tevez  37   Argentina  173 cm  75 kg   
2     15441          Abdisalam      Ibrahim  30      Norway  185 cm  71 kg   
3     17462              Jacob       Mellis  30     England  181 cm  69 kg   
4     17504               Adam       Buxton  29     England  185 cm  81 kg   
..      ...                ...          ...  ..         ...     ...    ...   
658   57182              Enric  Vallès Prat  32       Spain  190 cm  84 kg   
659  104522  Gabriel Sebastian        Tamaș  39     Romania  188 cm  79 kg   
660    2793   Guðlaugur Victor      Pálsson  31     Iceland  186 cm  77 kg   
661   18829               Jack     Wilshere  30     England  172 cm  68 kg   
662   48368             Martin       Hansen  32     Denmark  188 cm  85 kg   

    injured teamID         teamname  ... foulsDrawn foulsCommitted cardsYel  \
0     False     47        Tottenham  ...       None           None        1   
1     False     50  Manchester City  ...       None           None        6   
2     False     50  Manchester City  ...       None           None        0   
3     False     49          Chelsea  ...       None           None        0   
4     False     61            Wigan  ...       None           None        0   
..      ...    ...              ...  ...        ...            ...      ...   
658   False     54       Birmingham  ...       None           None        0   
659   False     60        West Brom  ...       None           None        4   
660   False     40        Liverpool  ...       None           None        0   
661   False     42          Arsenal  ...       None           None        7   
662   False     40        Liverpool  ...       None           None        0   

    cardsYelRed cardsRed penaltyWon penaltyCommited penaltyScored  \
0             0        0       None            None          None   
1             0        0       None            None          None   
2             0        0       None            None          None   
3             0        0       None            None          None   
4             0        0       None            None          None   
..          ...      ...        ...             ...           ...   
658           0        0       None            None          None   
659           0        1       None            None          None   
660           0        0       None            None          None   
661           0        1       None            None          None   
662           0        0       None            None          None   

    penaltyMissed penaltySaved  
0            None         None  
1            None         None  
2            None         None  
3            None         None  
4            None         None  
..            ...          ...  
658          None         None  
659          None         None  
660          None         None  
661          None         None  
662          None         None  

[663 rows x 46 columns]

In [31]:
gb = df.groupby(df['lastname']).count()#=='Kane'].goalAssists
df[df['teamname']=='Tottenham']

id        firstname                    lastname age  \
0       468           Vedran                     Ćorluka  35   
6     19289          Michael                      Dawson  38   
30   114808          William                      Gallas  37   
53   114817         Jermaine                       Jenas  38   
62    18928            Peter                      Crouch  40   
65    19683              Ben                     Alnwick  33   
100   27543   Wilson Roberto              Palacios Suazo  37   
195  114825            Carlo                    Cudicini  48   
207   90513           Steven                     Pienaar  39   
241   18789         Heurelho              da Silva Gomes  40   
255  114854         Jonathan                    Woodgate  36   
256  114865           Ledley                        King  33   
270  114799           Rafael               van der Vaart  35   
320   19178             Alan                      Hutton  35   
325   90503           Younes                      Kaboul  35   
328  104745             Niko                    Kranjčar  34   
372   46332  Bongani Sandile                     Khumalo  35   
399  104620            Roman                Pavlyuchenko  41   
420     167            Danny                        Rose  32   
430   18925            Aaron                      Lennon  35   
448   19215              Tom                 Huddlestone  36   
450   19318             Kyle                    Naughton  34   
457   22142      John Joseph                     Bostock  30   
460    1758    Jermain Colin                       Defoe  40   
491   30813           Sandro  Raniere Guimarães Cordeiro  33   
542     754             Luka                      Modrić  37   
562     758     Gareth Frank                        Bale  33   
608   18854           Andros                    Townsend  31   
639  105371          Giovani          dos Santos Ramírez  33   
651   19160             Jake                   Livermore  33   

             nationality  height weight injured teamID   teamname  ...  \
0                Croatia  192 cm  84 kg   False     47  Tottenham  ...   
6                England  189 cm  79 kg   False     47  Tottenham  ...   
30                France  183 cm  72 kg   False     47  Tottenham  ...   
53               England  179 cm  76 kg   False     47  Tottenham  ...   
62               England  201 cm  75 kg   False     47  Tottenham  ...   
65               England  188 cm  81 kg   False     47  Tottenham  ...   
100             Honduras  178 cm  71 kg   False     47  Tottenham  ...   
195                Italy  186 cm  79 kg   False     47  Tottenham  ...   
207         South Africa  170 cm  71 kg   False     47  Tottenham  ...   
241               Brazil  191 cm  91 kg   False     47  Tottenham  ...   
255              England  183 cm  80 kg   False     47  Tottenham  ...   
256              England  188 cm  86 kg   False     47  Tottenham  ...   
270          Netherlands  176 cm  74 kg   False     47  Tottenham  ...   
320             Scotland  185 cm  72 kg   False     47  Tottenham  ...   
325               France  193 cm  87 kg   False     47  Tottenham  ...   
328              Croatia  185 cm  80 kg   False     47  Tottenham  ...   
372         South Africa  188 cm  82 kg   False     47  Tottenham  ...   
399               Russia  188 cm  78 kg   False     47  Tottenham  ...   
420              England  173 cm  72 kg   False     47  Tottenham  ...   
430              England  165 cm  63 kg   False     47  Tottenham  ...   
448              England  188 cm  71 kg   False     47  Tottenham  ...   
450              England  180 cm  73 kg   False     47  Tottenham  ...   
457  Trinidad and Tobago  188 cm  84 kg   False     47  Tottenham  ...   
460              England  171 cm  65 kg   False     47  Tottenham  ...   
491               Brazil  187 cm  82 kg   False     47  Tottenham  ...   
542              Croatia  172 cm  66 kg   False     47  Tottenham  ...   
562                Wales  

In [32]:
df['season'] = '2009/2010'

In [33]:
#Export the df to a csv
df.to_csv('data2009&2010.csv')

In [34]:
counter

70